In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (8,8)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [5]:
#Please change to your name.
name = "Stanley Song"

#Please use this to control EDA. 
show_eda = False

In [66]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
171,1,0.668,0.570,0.079,0.841,0.871,0.268,0.304,0.455,0.416,...,0.776,0.275,0.747,0.859,0.630,0.935,0.352,0.540,0.344,0.769
175,0,0.727,0.462,0.184,0.176,0.915,0.991,0.575,0.324,0.762,...,0.473,0.802,0.308,0.747,0.248,0.230,0.014,0.907,0.182,0.435
154,0,0.410,0.138,0.492,0.752,0.189,0.665,0.567,0.807,0.432,...,0.899,0.063,0.411,0.287,0.586,0.840,0.812,0.135,0.137,0.157
94,1,0.091,0.446,0.506,0.891,0.228,0.183,0.109,0.895,0.030,...,0.769,0.104,0.336,0.365,0.794,0.900,0.587,0.945,0.301,0.877
172,1,0.172,0.362,0.280,0.002,0.596,0.972,0.060,0.083,0.340,...,0.142,0.807,0.758,0.508,0.137,0.553,0.581,0.106,0.388,0.907


In [67]:
df2 = pd.read_csv("testing.csv")
df2 = df2.drop(columns={"id"})
df2.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
8436,0,0.876,0.569,0.708,0.576,0.305,0.551,0.198,0.042,0.348,...,0.864,0.984,0.205,0.236,0.751,0.318,0.467,0.906,0.615,0.350
11836,0,0.001,0.245,0.528,0.572,0.945,0.746,0.276,0.828,0.929,...,0.067,0.268,0.668,0.895,0.621,0.165,0.555,0.755,0.727,0.034
3206,1,0.766,0.223,0.770,0.314,0.310,0.879,0.193,0.123,0.901,...,0.404,0.149,0.352,0.870,0.623,0.378,0.256,0.632,0.166,0.515
18864,0,0.815,0.712,0.402,0.123,0.771,0.833,0.319,0.778,0.841,...,0.999,0.741,0.446,0.969,0.174,0.178,0.493,0.005,0.785,0.438
13335,1,0.640,0.839,0.712,0.622,0.493,0.511,0.252,0.587,0.091,...,0.293,0.106,0.108,0.211,0.263,0.965,0.476,0.526,0.311,0.092


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [8]:
df.shape

(250, 201)

In [9]:
df2.shape

(19750, 201)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Columns: 201 entries, target to var_200
dtypes: float64(200), int64(1)
memory usage: 392.7 KB


Started with a basic DecisionTreeClassifier

In [18]:

y_train = np.array(df['target']).reshape(-1, 1)
x_train = np.array(df.drop(columns={'target'}))

y_test = np.array(df2['target']).reshape(-1, 1)
x_test = np.array(df2.drop(columns={'target'}))

pipeline = [
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier(max_depth=5, min_samples_split=3))
]

pipe = Pipeline(pipeline)

y_train = y_train.ravel()
y_test = y_test.ravel()

pipe.fit(x_train, y_train)

score = pipe.score(x_test, y_test)
print(score)

0.5479493670886076


Wasn't too accurate so I switched to a RandomForestClassifier and found better results

In [22]:

y_train = np.array(df['target']).reshape(-1, 1)
x_train = np.array(df.drop(columns={'target'}))

y_test = np.array(df2['target']).reshape(-1, 1)
x_test = np.array(df2.drop(columns={'target'}))

pipeline = [
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(criterion='entropy',
                                        max_depth=10, 
                                        min_samples_split=5,
                                        n_estimators=150))
]

pipe = Pipeline(pipeline)

y_train = y_train.ravel()
y_test = y_test.ravel()

pipe.fit(x_train, y_train)

score = pipe.score(x_test, y_test)
print(score)

0.6504303797468355


Built a gridsearch to help find better hyperparameters, although I found it too sometimes inaccurate

In [23]:
tree_para = {'min_samples_split':[2,3,4,5],
            'max_depth':[8,10,15,20,50],
            'n_estimators':[20,80,200,500],
            'criterion':["gini","entropy"]
            }

clf = GridSearchCV(estimator=RandomForestClassifier(), param_grid=tree_para, cv=5, n_jobs=-1)
clf.fit(x_train, y_train)
clf.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=10, min_samples_split=5,
                       n_estimators=80)

Using what I learned by testing out many different types and amounts I ended up with with these hyperparameters

In [54]:
y_train = np.array(df['target']).reshape(-1, 1)
x_train = np.array(df.drop(columns={'target'}))

y_test = np.array(df2['target']).reshape(-1, 1)
x_test = np.array(df2.drop(columns={'target'}))

pipeline = [
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(criterion='entropy', 
                                        max_depth=20,
                                        min_samples_leaf=4, 
                                        min_samples_split=6,
                                        n_estimators=500))
]

pipe = Pipeline(pipeline)

y_train = y_train.ravel()
y_test = y_test.ravel()

pipe.fit(x_train, y_train)

score = pipe.score(x_test, y_test)
print(score)

0.6845569620253165


Build a system to combine the pipeline and gridsearch method in one

In [15]:
y_train = np.array(df['target']).reshape(-1, 1)
x_train = np.array(df.drop(columns=['target']))

y_test = np.array(df2['target']).reshape(-1, 1)
x_test = np.array(df2.drop(columns=['target']))


pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier())
])

# Parameters 
tree_para = {
    'classifier__min_samples_split': [2, 3, 4, 5],
    'classifier__max_depth': [8, 10, 15, 25, 40],
    'classifier__n_estimators': [200, 300, 400, 500, 600],
    'classifier__criterion': ["gini", "entropy"]   
}

# Use GridSearch method to find best parameters
pipe = GridSearchCV(estimator=pipeline, param_grid=tree_para, cv=5, n_jobs=-1)

y_train = y_train.ravel()
y_test = y_test.ravel()

pipe.fit(x_train, y_train)  

best_estimator = pipe.best_estimator_

score = best_estimator.score(x_test, y_test)  
print("Best score:", score)
print("Best parameters:", best_estimator)


Best score: 0.6671392405063291
Best parameters: Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(max_depth=8, min_samples_split=3,
                                        n_estimators=400))])


### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [58]:
# My best Model

Y_train = np.array(df['target']).reshape(-1, 1)
X_train = np.array(df.drop(columns={'target'}))

Y_test = np.array(df2['target']).reshape(-1, 1)
X_test = np.array(df2.drop(columns={'target'}))

pipeline = [
    ('scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(criterion='entropy', 
                                        max_depth=20,
                                        min_samples_leaf=4, 
                                        min_samples_split=3,
                                        n_estimators=500))
]

best = Pipeline(pipeline)

Y_train = Y_train.ravel()
Y_test = Y_test.ravel()

best.fit(X_train, Y_train)

print(best.score(X_test, Y_test))
print(best)

0.6753924050632911
Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 RandomForestClassifier(criterion='entropy', max_depth=20,
                                        min_samples_leaf=4, min_samples_split=3,
                                        n_estimators=500))])


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [ ]:
#Load Test Data
test_df = pd.read_csv("testing.csv")
test_df = test_df.drop(columns={"id"})
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:
<ul>
<li> I put in some hyperparameters to reduce the overfitting done and tried to find the best hyperparameters by using a gridsearch to slove for the optimal slection of parameters given. 
<li> In addition, to improve accuracy I used cross validation to better evaluate our limited training data
</ul>